In [ ]:

import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px 
from plotly.subplots import make_subplots
import plotly.graph_objects as go


df = pd.read_csv('incidents.csv')



def cast_date_str_to_date_and_sort(df):
    df['DATE'] = pd.to_datetime(df['DATE'])
    df.sort_values(by='DATE', inplace=True)
    new_date_range_filled = pd.date_range(start='2021-01-01', end='2023-01-01', freq='MS')
    df.set_index("DATE", inplace=True)
    new_index = pd.Index(new_date_range_filled, name="DATE")
    df = df.reindex(new_index, fill_value=0)
    df.reset_index(inplace=True)
    df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime('%Y-%m')
    
    return df
    
    
def get_num_of_incidents_for_month_dataframe(df):
    date_with_num_of_incidents_df = df.groupby('DATE').size().reset_index(name='num_of_incidents')
    date_with_num_of_incidents_df = cast_date_str_to_date_and_sort(date_with_num_of_incidents_df)
    print(date_with_num_of_incidents_df)
    
    return date_with_num_of_incidents_df

    
def draw_line_chart_for_num_of_incidents_with_months(month_and_num_of_incidents, num_of_inc, name_prefix=""):
    x = month_and_num_of_incidents['DATE']
    y = month_and_num_of_incidents[num_of_inc]

    plt.plot(x, y)
    plt.title(f'{name_prefix}Number of incidents from 2021 to 2023')
    plt.xlabel('Month')
    plt.ylabel('Number of incidents')
    plt.xticks(rotation=90)
    plt.show()

    
def get_num_of_incidents_for_month_for_each_country():
    gb_countries = df.groupby('COUNTRY')
    dataframes = []
    for country, datafr in gb_countries:
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name=country)
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)
        dataframes.append(date_and_num_of_incidents)
    
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['DATE'], how='outer'), dataframes).fillna('0')
    print(df_merged)
    return df_merged
    

def draw_stacked_bar(df):
    dates = df['DATE']
    country_names = list(df.columns.values)
    country_names.remove('DATE')

    countries = [df[country_name] for country_name in country_names]
    bottom = 0
    for i, country in enumerate(countries):
        plt.bar(dates, country, bottom=bottom, label=country_names[i])
        bottom+=country
        
    plt.xticks(rotation=90)
    plt.xlabel('Months')
    plt.ylabel('Number of incidents')
    plt.show()


def draw_charts_for_each_country_with_num_of_incidents(df):
    gb_countries = df.groupby('COUNTRY')
    for country, datafr in gb_countries:
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name=f"{country}_num_of_incidents")
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)
        draw_line_chart_for_num_of_incidents_with_months(date_and_num_of_incidents, f"{country}_num_of_incidents", f"{country}: ")

  
def draw_multiple_line_charts_in_1_figure(dataframe, column_to_gb):
    gb_countries = df.groupby(column_to_gb)
    num_of_rows = len(gb_countries)//2+1
    fig, axs = plt.subplots(num_of_rows, 2)
    fig.suptitle('Number of incidents amongst countries')
    
    if len(gb_countries)%2:
        axs[num_of_rows-1, 1].remove()
    
    for i, (country, datafr) in enumerate(gb_countries):
        row_num = int(i/2)
        side = i%2
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name='num_of_incidents')
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)
        x = date_and_num_of_incidents['DATE']
        y = date_and_num_of_incidents['num_of_incidents']
        current_ax = axs[row_num][side]
        current_ax.plot(x, y)
        current_ax.set_ylabel('Incidents', fontsize = 8.0) # Y label
        current_ax.set_xlabel('Months', fontsize = 8.0) # Y label
        current_ax.tick_params(labelrotation=90)
        current_ax.title.set_text(f"{country}: Number of incidents")
    
    plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=1.5,
                    top=0.9,
                    wspace=0.5,
                    hspace=1.8)
    plt.show()



#plotly chart generation


def draw_plotly_line_chart(df):
    fig = px.line(df, x="DATE", y="num_of_incidents", title="Number of incidents", labels={'DATE':'Months', 'num_of_incidents':'Num of incidents'}) 
  
    fig.update_layout(
        title_font_family="Times New Roman",
        title_font_color="red",
        legend_title_font_color="green"
    )
    fig.show()

    
def draw_plotly_stacked_bar(df):
    country_names = list(df.columns.values)
    country_names.remove('DATE')
    fig = px.bar(df, x="DATE", y=country_names, title="Plotly Bar Chart")

    fig.show()
    

def draw_plotly_subplots(dataframe, column_to_gb):
    gb_countries = df.groupby(column_to_gb)
    num_of_rows = len(gb_countries)//2+1
    fig = make_subplots(rows=num_of_rows, cols=2, subplot_titles=[f"{country}: num of incidents" for country, df in gb_countries])
    for i, (country, datafr) in enumerate(gb_countries):
        row_num = (i//2)+1
        side = (i%2)+1
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name='num_of_incidents')
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)
        x = date_and_num_of_incidents['DATE']
        y = date_and_num_of_incidents['num_of_incidents']
        fig.add_trace(
            go.Scatter(x=x, y=y, name=f"{country}: num of incidents"),
            row=row_num, col=side
        )
    fig.update_layout(height=1000, width=1000, title_text="Num of incidents in each countries")
    fig.show()
    
    
def _create_plotly_fig_for_all_incidents_with_moving_average(df_all):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x = df_all['DATE'],
            y = df_all['num_of_incidents'],
            name = 'incidents'
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x = df_all['DATE'],
            y = df_all['moving_average'],
            name = 'moving average'
        )
    )
    
    return fig


def _create_country_selector_buttons(df_all, df):
    country_buttons = [dict(args=[{"y":[df_all['num_of_incidents'], df_all['moving_average']], 'x':[df_all['DATE']]}, {'title': 'Incidents in all countries'}],
                label="ALL",
                method="update") 
                ]
        
    for country, datafr in df.groupby('COUNTRY'):
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name="num_of_incidents")
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)    
        date_and_num_of_incidents['moving_average'] = date_and_num_of_incidents['num_of_incidents'].rolling(3).mean()
        button = dict(
            args=[{"y":[date_and_num_of_incidents['num_of_incidents'], date_and_num_of_incidents['moving_average']], 'x':[date_and_num_of_incidents['DATE']]}, {'title':f"Incidents in {country}"}],
                label=country,
                method="update")
        country_buttons.append(button)
    return country_buttons
    
    
def _create_restyle_buttons():
    restyle_buttons = [dict(
                    args=["type", "line"],
                    label="Line chart",
                    method="restyle"
                ),
                 dict(
                    args=["type", "bar"],
                    label="Bar chart",
                    method="restyle"
                )
            ]
    return restyle_buttons


def _create_show_moving_average_buttons():
    show_moving_average_buttons = [dict(
            args=[{'visible': [True, True]}],
                label='Show moving average',
                method="update"),
                dict(
                    args=[{'visible': [True, False]}],
                        label='Do not show moving average',
                        method="update")]
    return show_moving_average_buttons


def draw_plotly_charts_with_menu(df):
    df_all = get_num_of_incidents_for_month_dataframe(df)
    df_all['moving_average'] = df_all['num_of_incidents'].rolling(3).mean()
    
    fig = _create_plotly_fig_for_all_incidents_with_moving_average(df_all)

    country_buttons = _create_country_selector_buttons(df_all, df)
    show_moving_average_buttons = _create_show_moving_average_buttons()
    restyle_buttons = _create_restyle_buttons()
    
    fig.update_layout(updatemenus=[dict(buttons=country_buttons, x=1.30, y=1, xanchor='left', yanchor='top'),
                                   dict(buttons=show_moving_average_buttons, x=1.30, y=0.85, xanchor='left', yanchor='top'),
                                   dict(buttons=restyle_buttons, x=1.30, y=0.70, xanchor='left', yanchor='top')])


    fig.show()

    

    
    
#task 1   
number_of_incidents_for_month = get_num_of_incidents_for_month_dataframe(df)
draw_line_chart_for_num_of_incidents_with_months(number_of_incidents_for_month, "num_of_incidents")
draw_plotly_line_chart(number_of_incidents_for_month)
#task 2
dataframe = get_num_of_incidents_for_month_for_each_country()
draw_stacked_bar(dataframe)
draw_plotly_stacked_bar(dataframe)
#task 3
draw_charts_for_each_country_with_num_of_incidents(df)
# task 4
draw_multiple_line_charts_in_1_figure(df, 'COUNTRY')
draw_plotly_subplots(df, 'COUNTRY')
#task 5
draw_plotly_charts_with_menu(df)



